In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from google.colab import files
import joblib

import warnings
warnings.filterwarnings("ignore")

In [ ]:
#!pip install -U gdown

In [ ]:

import gdown
file_id = "1ini0-kUwgXoVaWALnOtVWoTRmONh_8qr"
url = f"https://drive.google.com/uc?id={file_id}"
output = "AzureFunctionsInvocationTraceForTwoWeeksJan2021.csv"

gdown.download(url, output, quiet=False)

In [ ]:
df = pd.read_csv("AzureFunctionsInvocationTraceForTwoWeeksJan2021.csv")

In [ ]:
df.head()

In [ ]:
initial_rows = df.shape[0]
print(f"Rows before pre-processing = {initial_rows}")

In [ ]:
df.info()

Pre-processing and analyzing the dataset

In [ ]:
df_cp = df.copy()

In [ ]:
df_cp.isnull().sum()

In [ ]:
#Unique entities
print(f"Applications: {df['app'].nunique():,}")
print(f"Functions: {df['func'].nunique():,}")
print(f"App-Function pairs: {df.groupby(['app', 'func']).ngroups:,}")

In [ ]:
#Converting timestamp to datetime
df_cp['timestamp'] = pd.to_datetime(df_cp['end_timestamp'], unit = 's')

In [ ]:
#Time range
duration_hours = (df_cp['timestamp'].max() - df_cp['timestamp'].min()).total_seconds() / 3600
print(f"  Duration: {duration_hours:.1f} hours ({duration_hours/24:.1f} days)")

In [ ]:
#Removing invalid duration
before_duration_clean = len(df_cp)
df_cp = df_cp[(df_cp['duration'] > 0) & (df_cp['duration'] < 3600)]
print(f"   Removed {before_duration_clean - len(df_cp):,} rows with invalid durations")


In [ ]:
#Checking extreme outliers
sns.boxplot(data=df_cp, y = 'duration')

In [ ]:
#Removing extreme outliers
before_outliers = len(df_cp)
Q1 = df_cp['duration'].quantile(0.25)
Q3 = df_cp['duration'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 3* IQR
upper_bound = Q3 + 3* IQR
df_cp = df_cp[(df_cp['duration']>=max(0, lower_bound)) & (df_cp['duration']<=upper_bound)]
print(f"Removed {before_outliers - len(df_cp):,} extreme outliers")

In [ ]:
#Sorting based on timestamp
df_cp = df_cp.sort_values('timestamp').reset_index(drop=True)

In [ ]:
processed_rows = df_cp.shape[0]
print(f"Length of processed dataset ={processed_rows}")

In [ ]:
#Data Retention Rate
print(f"Data retention rate = {processed_rows/initial_rows *100:.2f}% ")

Feature Engineering & Analysing the dataset

In [ ]:
df_cp['hour'] = df_cp['timestamp'].dt.hour
df_cp['dayofweek'] = df_cp['timestamp'].dt.dayofweek
df_cp['dayofmonth'] = df_cp['timestamp'].dt.day
df_cp['month'] = df_cp['timestamp'].dt.month
df_cp['is_weekend'] = df_cp['dayofweek'].isin([5,6]).astype(int)

In [ ]:
#Identifying cold starts
df_cp = df_cp.sort_values(['app', 'func','timestamp'])
df_cp['time_gap'] = df_cp.groupby(['app','func'])['timestamp'].diff().dt.total_seconds()
cold_start_threshold = 600
df_cp['is_potential_coldstart'] = ((df_cp['time_gap']>cold_start_threshold) | (df_cp['time_gap'].isna())).astype(int)


In [ ]:
#Function STatistics Analysis

func_stats = df_cp.groupby(['app', 'func']).agg({
    'duration':['count', 'mean', 'std', 'min', 'max'],
    'is_potential_coldstart':['sum', 'mean'],
    'time_gap': ['mean'],
    'timestamp':['min','max']
}).reset_index()


# Flatten column names
func_stats.columns = [
        'app', 'func', 'invocation_count', 'avg_duration', 'std_duration',
        'min_duration', 'max_duration', 'cold_starts', 'cold_start_rate',
        'avg_time_gap', 'first_invocation', 'last_invocation'
]

func_stats['activity_span_hours'] = (func_stats['last_invocation'] - func_stats['first_invocation']).dt.total_seconds()/3600
func_stats['invocations_per_hour'] = func_stats['invocation_count']/np.maximum(func_stats['activity_span_hours'],0.1)
func_stats['duration_variability'] = func_stats['std_duration']/np.maximum(func_stats['avg_duration'], 0.001)



In [ ]:
func_stats = func_stats.sort_values('invocation_count', ascending = False)

In [ ]:
print(f"Total unique functions: {len(func_stats):,}")
print(f"Functions with >1000 invocations: {(func_stats['invocation_count'] > 1000).sum()}")
print(f"Functions with >100 invocations: {(func_stats['invocation_count'] > 100).sum()}")
print(f"Functions with >10 invocations: {(func_stats['invocation_count'] > 10).sum()}")

In [ ]:
#Top 10 most active functions:
for i, (_, row) in enumerate(func_stats.head(10).iterrows()):
  print(f"\n{i+1}. Function: {row['func']}")
  print(f"Invocations: {row['invocation_count']:,}")
  print(f"Avg duration: {row['avg_duration']:.3f}s")
  print(f"Cold start rate: {row['cold_start_rate']*100:.1f}%")
  print(f"Activity span: {row['activity_span_hours']:.1f} hours")
  print(f"Invocations/hour: {row['invocations_per_hour']:.2f}")

In [ ]:
#Invocation statistics
print(f"Total invocations = {len(df_cp):,}")
print(f"Total invocations per hour = {len(df_cp)/duration_hours:.0f}")
print(f"Total invocations per function = {len(df_cp)/df_cp['func'].nunique():.0f}")

In [ ]:
#Duration Analysis
duration_stats = df_cp['duration'].describe()
print("Duration statistics (seconds):")
for stat, value in duration_stats.items():
  print(f"{stat} : {value:.4f}")

In [ ]:
#Duration Percentile
percentiles = [90,95,99,99.9]
for p in percentiles:
  value = df_cp['duration'].quantile(p/100)
  print(f"{p}th percentile: {value:.4f}")

In [ ]:
#Duration Categories
short_duration = (df_cp['duration']<=0.1).sum()
medium_duration = ((df_cp['duration']>0.1) & (df_cp['duration']<=1.0)).sum()
long_duration = (df_cp['duration']>1.0).sum()
print(f"Very fast (<0.1s) = {short_duration:,} ({short_duration/len(df_cp)*100:.1f}%)")
print(f"Medium (0.1s - 1.0s) = {medium_duration:,} ({medium_duration/len(df_cp)*100:.1f}%)")
print(f"Slow (>1.0s) = {long_duration:,} ({long_duration/len(df_cp)*100:.1f}%)")

In [ ]:
#Cold start analysis
total_invocations = len(df_cp)
cold_starts = df_cp['is_potential_coldstart'].sum()
cold_start_rate = cold_starts/total_invocations *100
print(f"Total invocations: {total_invocations:,}")
print(f"Potential cold starts: {cold_starts:,}")
print(f"Cold start rate: {cold_start_rate:.2f}%")

In [ ]:
# Cold start by time gap analysis
time_gaps = df_cp[df_cp['time_gap'].notna()]['time_gap']
gap_stats = time_gaps.describe()
for stat, value in gap_stats.items():
  if stat in ['mean', 'std', 'min', 'max']:
    print(f"{stat} : {value:,.0f} seconds ({value/60:.1f} minutes)")

In [ ]:
# Cold start vs warm start duration comparison
cold_duration = df_cp[df_cp['is_potential_coldstart'] == 1]['duration'].mean()
warm_duration = df_cp[df_cp['is_potential_coldstart'] == 0]['duration'].mean()

print(f"Cold start avg duration: {cold_duration:.4f}s")
print(f"Warm start avg duration: {warm_duration:.4f}s")
print(f"Cold start overhead: {cold_duration - warm_duration:.4f}s ({(cold_duration/warm_duration-1)*100:.1f}% increase)")


In [ ]:
df_cp.head()

In [ ]:
#Pattern Analysis
hourly_stats = df_cp.groupby('hour').agg({
    'func':'count',
    'duration': 'mean',
    'is_potential_coldstart':['sum','mean']
})
hourly_stats.columns = ['invocations', 'avg_duration', 'cold_starts', 'cold_start_rate']

In [ ]:
hourly_stats.head()

In [ ]:
#Hourly patterns
peak_hour = hourly_stats['invocations'].idxmax()
quiet_hour = hourly_stats['invocations'].idxmin()
peak_coldstart_hour = hourly_stats['cold_start_rate'].idxmax()

print(f"Peak hour: {peak_hour}:00 ({hourly_stats.loc[peak_hour, 'invocations']:,} invocations)")
print(f"Quietest hour: {quiet_hour}:00 ({hourly_stats.loc[quiet_hour, 'invocations']:,} invocations)")
print(f"Cold Start Hour: {peak_coldstart_hour}:00 ({hourly_stats.loc[peak_coldstart_hour, 'cold_start_rate']*100:.1f}% invocations)")

In [ ]:
#Daily Patterns
daily_stats = df_cp.groupby('dayofweek').agg({
    'func' : 'count',
    'duration' : 'mean',
    'is_potential_coldstart' : ['sum', 'mean']
})
daily_stats.columns = ['invocations', 'avg_duration','cold_starts', 'cold_start_rate']

In [ ]:
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
for i, day in enumerate(days):
  if i in daily_stats.index:
    invocations = daily_stats.loc[i, 'invocations']
    coldstart_rate = daily_stats.loc[i, 'cold_start_rate'] * 100
    print(f"{day}: {invocations:,} invocations, {coldstart_rate:.1f}% cold starts")


In [ ]:
# Weekend vs Weekday
weekend_comparison = df_cp.groupby('is_weekend').agg({
        'func': 'count',
        'duration': 'mean',
        'is_potential_coldstart': 'mean'
})
weekend_comparison.columns = ['invocations', 'avg_duration', 'cold_start_rate']

print(f"\nWeekend vs Weekday comparison:")
print(f"Weekdays: {weekend_comparison.loc[0, 'invocations']:,} invocations, {weekend_comparison.loc[0, 'cold_start_rate']*100:.1f}% cold starts")
print(f"Weekends: {weekend_comparison.loc[1, 'invocations']:,} invocations, {weekend_comparison.loc[1, 'cold_start_rate']*100:.1f}% cold starts")



Visualizations


In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Azure Functions Dataset Analysis', fontsize=16, fontweight='bold')

#hourly traffic pattern(plot 1)
axes[0, 0].plot(hourly_stats.index, hourly_stats['invocations'],
                marker='o', linewidth=2, markersize=6, color='blue')
axes[0, 0].set_title('Traffic by Hour of Day', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Hour')
axes[0, 0].set_ylabel('Number of Invocations')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].set_xticks(range(0, 24, 4))

# Daily traffic pattern (Plot 2)
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
axes[0, 1].bar(range(len(daily_stats)), daily_stats['invocations'],
               color='skyblue', alpha=0.8)
axes[0, 1].set_title('Traffic by Day of Week', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Day')
axes[0, 1].set_ylabel('Number of Invocations')
axes[0, 1].set_xticks(range(len(days)))
axes[0, 1].set_xticklabels(days)

#Duration distribution
axes[0, 2].hist(df['duration'], bins=50, alpha=0.7, color='green', edgecolor='black')
axes[0, 2].set_title('Function Duration Distribution', fontsize=12, fontweight='bold')
axes[0, 2].set_xlabel('Duration (seconds)')
axes[0, 2].set_ylabel('Frequency')
axes[0, 2].set_yscale('log')

#Cold start rate by hour
axes[1, 0].plot(hourly_stats.index, hourly_stats['cold_start_rate'] * 100,
           marker='s', color='red', linewidth=2, markersize=5)
axes[1, 0].set_title('Cold Start Rate by Hour', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Hour')
axes[1, 0].set_ylabel('Cold Start Rate (%)')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].set_xticks(range(0, 24, 4))

#Top functions by invocations
top_funcs = df['func'].value_counts().head(10)
axes[1, 1].barh(range(len(top_funcs)), top_funcs.values,
                color='orange', alpha=0.7)
axes[1, 1].set_title('Top 10 Functions by Invocation Count', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Number of Invocations')
axes[1, 1].set_yticks(range(len(top_funcs)))
axes[1, 1].set_yticklabels([f"...{func[-12:]}" for func in top_funcs.index])

#Cold start vs warm start duration comparison
cold_durations = df_cp[df_cp['is_potential_coldstart'] == 1]['duration']
warm_durations = df_cp[df_cp['is_potential_coldstart'] == 0]['duration']

axes[1, 2].hist([warm_durations, cold_durations], bins=30, alpha=0.7,
                    label=['Warm Start', 'Cold Start'], color=['blue', 'red'])
axes[1, 2].set_title('Duration: Cold Start vs Warm Start', fontsize=12, fontweight='bold')
axes[1, 2].set_xlabel('Duration (seconds)')
axes[1, 2].set_ylabel('Frequency')
axes[1, 2].legend()
axes[1, 2].set_yscale('log')

plt.tight_layout()
plt.show()

plt.show()



In [ ]:
#Time gap distribution analysis
time_gaps = df_cp[df_cp['time_gap'].notna()]['time_gap'] #first invocation of each function

print(f"  Total gaps analyzed: {len(time_gaps):,}")
print(f"  Mean gap: {time_gaps.mean():.0f} seconds ({time_gaps.mean()/60:.1f} minutes)")
print(f"  Median gap: {time_gaps.median():.0f} seconds ({time_gaps.median()/60:.1f} minutes)")


In [ ]:
# Gap categories
categories = {
        'Very short (≤10s)': (time_gaps <= 10).sum(),
        'Short (10s-1min)': ((time_gaps > 10) & (time_gaps <= 60)).sum(),
        'Medium (1-10min)': ((time_gaps > 60) & (time_gaps <= 600)).sum(),
        'Long (10min-1hr)': ((time_gaps > 600) & (time_gaps <= 3600)).sum(),
        'Very long (>1hr)': (time_gaps > 3600).sum()
}

for category, count in categories.items():
  percentage = (count / len(time_gaps)) * 100
  print(f"  {category}: {count:,} ({percentage:.1f}%)")

In [ ]:
#time gap distribution
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.hist(time_gaps, bins=50, alpha=0.7, color='purple', edgecolor='black')
plt.xlabel('Time Gap (seconds)')
plt.ylabel('Frequency')
plt.title('Time Gap Distribution (All)')
plt.yscale('log')

plt.subplot(1, 2, 2)
# Focusing on gaps up to 1 hour for better visibility
short_gaps = time_gaps[time_gaps <= 3600]
plt.hist(short_gaps, bins=50, alpha=0.7, color='purple', edgecolor='black')
plt.xlabel('Time Gap (seconds)')
plt.ylabel('Frequency')
plt.title('Time Gap Distribution (≤1 hour)')
plt.axvline(x=600, color='red', linestyle='--', linewidth=2, label='Cold Start Threshold')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:

# Filter functions with reasonable activity (>50 invocations)
active_funcs = func_stats[func_stats['invocation_count'] >= 50].copy()

In [ ]:
active_funcs.head()

In [ ]:
active_funcs['regularity_score'] = 1 / (1 + active_funcs['duration_variability'])  # Higher = more regular
active_funcs['activity_intensity'] = active_funcs['invocations_per_hour']

In [ ]:
active_funcs.describe()

In [ ]:
# Classifying workloads
conditions = [
        (active_funcs['cold_start_rate'] > 0.3) & (active_funcs['activity_intensity'] < 15),       # Sporadic (High cold start, Low intensity)
        (active_funcs['cold_start_rate'] < 0.1) & (active_funcs['activity_intensity'] > 1000),      # Steady High (Low cold start, Very High intensity)
        (active_funcs['activity_intensity'] > 30) & (active_funcs['regularity_score'] > 0.5),      # Regular (Moderate/High intensity AND predictable)
        (active_funcs['cold_start_rate'] > 0.15) & (active_funcs['duration_variability'] > 2),     # Bursty (Medium cold start, High variability)
    ]

In [ ]:
choices = ['Sporadic', 'Steady-High', 'Regular', 'Bursty']
active_funcs['workload_type'] = np.select(conditions, choices, default='Mixed')

workload_summary = active_funcs['workload_type'].value_counts()
print(f"Workload type distribution (functions with ≥50 invocations):")
for workload_type, count in workload_summary.items():
  percentage = (count / len(active_funcs)) * 100
  print(f"  {workload_type}: {count} functions ({percentage:.1f}%)")

In [ ]:
#workload type example
for workload_type in ['Sporadic', 'Steady-High', 'Regular', 'Bursty', 'Mixed']:
  if workload_type in active_funcs['workload_type'].values:
    example = active_funcs[active_funcs['workload_type'] == workload_type].iloc[0]
    print(f"\n{workload_type} example:")
    print(f"Function: {example['func'][:20]}...")
    print(f"Invocations: {example['invocation_count']:,}")
    print(f"Cold start rate: {example['cold_start_rate']*100:.1f}%")
    print(f"Invocations/hour: {example['activity_intensity']:.1f}")
    print(f"Duration variability: {example['duration_variability']:.2f}")


In [ ]:
df_features = df_cp.sort_values(['app', 'func', 'timestamp'])

print("Creating cyclical time features...")
# Cyclical encoding for time features (better for ML models)
df_features['hour_sin'] = np.sin(2 * np.pi * df_features['hour'] / 24)
df_features['hour_cos'] = np.cos(2 * np.pi * df_features['hour'] / 24)
df_features['dayofweek_sin'] = np.sin(2 * np.pi * df_features['dayofweek'] / 7)
df_features['dayofweek_cos'] = np.cos(2 * np.pi * df_features['dayofweek'] / 7)
df_features['dayofmonth_sin'] = np.sin(2 * np.pi * df_features['dayofmonth'] / 31)
df_features['dayofmonth_cos'] = np.cos(2 * np.pi * df_features['dayofmonth'] / 31)


In [ ]:
# Business hours and weekend patterns
df_features['is_business_hours'] = ((df_features['hour'] >= 9) &
                                      (df_features['hour'] <= 17) &
                                       (df_features['is_weekend'] == 0)).astype(int)
df_features['is_peak_hours'] = ((df_features['hour'] >= 10) &
                                   (df_features['hour'] <= 15)).astype(int)
df_features['is_evening'] = ((df_features['hour'] >= 18) &
                                (df_features['hour'] <= 23)).astype(int)

In [ ]:
df_features = df_features.sort_values(by=['app', 'func', 'timestamp'])

# Calculate Rolling Features

windows = [5, 15, 30]
for window in windows:
  # Rolling mean of durations
  df_features[f'duration_rolling_mean_{window}'] = (
      df_features.groupby(['app', 'func'])['duration']
      .rolling(window=window, min_periods=1)
      .mean()
      .reset_index(level=[0, 1], drop=True) #to drop the new app and func levels created by groupby
  )

  # Rolling std of durations
  df_features[f'duration_rolling_std_{window}'] = (
      df_features.groupby(['app', 'func'])['duration']
      .rolling(window=window, min_periods=1)
      .std()
      .fillna(0)
      .reset_index(level=[0, 1], drop=True)
  )

  # Rolling count (frequency)
  df_features[f'frequency_{window}'] = (
      df_features.groupby(['app', 'func'])['duration']
      .rolling(window=window, min_periods=1)
      .count()
      .reset_index(level=[0, 1], drop=True)
  )

print("Creating lag features...")
#Lag features
lags = [1, 2, 3, 5]
for lag in lags:
    df_features[f'duration_lag_{lag}'] = (
        df_features.groupby(['app', 'func'])['duration']
        .shift(lag)
        .fillna(df_features['duration'].mean())
    )

In [ ]:
# Inter-arrival time features
df_features['time_since_last'] = df_features.groupby(['app', 'func'])['timestamp'].diff().dt.total_seconds().fillna(0)
df_features['time_since_last_log'] = np.log1p(df_features['time_since_last'])

# Inverse frequency (how rare are invocations for this function)
df_features['inv_frequency'] = 1.0 / (df_features.groupby(['app', 'func']).cumcount() + 1)

print("Creating burst detection features...")
# Burst detection
df_features['is_burst'] = (df_features['time_since_last'] < 10).astype(int)  # Calls within 10 seconds
df_features['burst_size'] = df_features.groupby(['app', 'func'])['is_burst'].rolling(10, min_periods=1).sum().reset_index(level=[0, 1], drop=True)


In [ ]:
df_features.info()

In [ ]:
df_features.head()

Feature Corelation Analysis

In [ ]:
# Selecting numeric columns for correlation analysis
numeric_cols = df_features.select_dtypes(include = [np.number]).columns

In [ ]:
# Removing identifier columns
feature_cols = [col for col in numeric_cols
                   if col not in ['app', 'func'] and 'timestamp' not in col.lower()]

In [ ]:
# Computing correlation matrix
correlation_matrix = df_features[feature_cols].corr()

In [ ]:
# Finding highly correlated feature pairs
high_corr_pairs = []
# Looping through the upper triangle of the matrix to find pairs
for i in range(len(correlation_matrix.columns)):
  for j in range(i + 1, len(correlation_matrix.columns)):

    corr_val = correlation_matrix.iloc[i, j]
    if abs(corr_val) > 0.7:  # High correlation threshold
      high_corr_pairs.append((
      correlation_matrix.columns[i],
      correlation_matrix.columns[j],
      corr_val
      ))

In [ ]:


# Visualize correlation heatmap for key features
key_features = [
        'duration', 'is_potential_coldstart', 'time_gap', 'hour', 'dayofweek',
        'is_business_hours', 'duration_rolling_mean_15', 'frequency_15',
        'time_since_last_log', 'burst_size'
]

available_features = [f for f in key_features if f in df_features.columns]

plt.figure(figsize=(12, 10))
sns.heatmap(df_features[available_features].corr(),
                annot=True, cmap='coolwarm', center=0,
                square=True, fmt='.2f')
plt.title('Feature Correlation Matrix (Key Features)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

Creating Time Series Dataset for Top Functions

In [ ]:
# Selecting top functions with sufficient data
top_functions = func_stats.head(10)
time_series_datasets = {}

for idx, row in top_functions.iterrows():
  app_id = row['app']
  func_id = row['func']

  print(f"\nProcessing function {idx+1}/10: {func_id}...")
  print(f"  Total invocations: {row['invocation_count']:,}")

  # Get data for this function
  func_data = df_features[
  (df_features['app'] == app_id) &
  (df_features['func'] == func_id)
  ].copy()

  # Create minute-level time series
  func_data['timestamp_minute'] = func_data['timestamp'].dt.floor('1min')

  # Aggregate by minute
  minute_series = func_data.groupby('timestamp_minute').agg({
           'duration': ['count', 'mean', 'std'],
            'is_potential_coldstart': 'sum',
            'hour': 'first',
            'dayofweek': 'first',
            'is_business_hours': 'first',
            'is_weekend': 'first'
        }).reset_index()

  # Flatten column names
  minute_series.columns = [
            'ds', 'y', 'avg_duration', 'std_duration',
            'cold_starts', 'hour', 'dayofweek',
            'is_business_hours', 'is_weekend'
        ]

  # Fill missing minutes with zeros
  start_time = minute_series['ds'].min()
  end_time = minute_series['ds'].max()
  complete_range = pd.date_range(start=start_time, end=end_time, freq='1min')

  minute_series = minute_series.set_index('ds').reindex(complete_range, fill_value=0)
  minute_series.index.name = 'ds'
  minute_series = minute_series.reset_index()

  # Forward fill categorical features
  minute_series['hour'] = minute_series['ds'].dt.hour
  minute_series['dayofweek'] = minute_series['ds'].dt.dayofweek
  minute_series['is_business_hours'] = ((minute_series['hour'] >= 9) &
                                             (minute_series['hour'] <= 17) &
                                             (minute_series['dayofweek'] < 5)).astype(int)
  minute_series['is_weekend'] = minute_series['dayofweek'].isin([5, 6]).astype(int)

  # Store dataset
  dataset_info = {
            'data': minute_series,
            'function_id': func_id + '...',
            'app_id': app_id + '...',
            'total_invocations': row['invocation_count'],
            'data_points': len(minute_series),
            'activity_hours': row['activity_span_hours'],
            'cold_start_rate': row['cold_start_rate']
        }

  time_series_datasets[f'func_{idx+1}'] = dataset_info

  print(f"Time series length: {len(minute_series):,} minutes")
  print(f"Non-zero periods: {(minute_series['y'] > 0).sum():,}")
  print(f"Coverage: {(minute_series['y'] > 0).mean()*100:.1f}%")


In [ ]:

print(f"\n Created time series for {len(time_series_datasets)} functions")

Export Data for ML Training

In [ ]:
# Export full processed dataset
print("Exporting full processed dataset")
df_features.to_csv('azure_functions_processed_full.csv', index=False)
print(f"   Saved: azure_functions_processed_full.csv ({df_features.shape[0]:,} rows)")


In [ ]:
# Export individual function time series for Prophet/LSTM
print("Exporting individual function time series...")
for func_name, dataset in time_series_datasets.items():
  filename = f'timeseries_{func_name}.csv'
  dataset['data'].to_csv(filename, index=False)
  print(f"   Saved: {filename} ({len(dataset['data']):,} data points)")

In [ ]:
# Export summary statistics
print("Creating summary report...")
summary_data = []
for func_name, dataset in time_series_datasets.items():
  data = dataset['data']
  summary_data.append({
            'function_name': func_name,
            'function_id': dataset['function_id'],
            'total_invocations': dataset['total_invocations'],
            'time_series_length': len(data),
            'non_zero_periods': (data['y'] > 0).sum(),
            'coverage_pct': (data['y'] > 0).mean() * 100,
            'max_requests_per_minute': data['y'].max(),
            'avg_requests_per_minute': data['y'].mean(),
            'cold_start_rate_pct': dataset['cold_start_rate'] * 100
        })

In [ ]:
summary_df = pd.DataFrame(summary_data)
summary_df.to_csv('dataset_summary.csv', index=False)
print(f"   Saved: dataset_summary.csv")

print(f"\n Data export completed!")
print(f"Ready for Prophet/LSTM model training on {len(time_series_datasets)} functions")


In [ ]:
#Final Analysis Summary
print("Dataset Processing Summary:")
print(f"Original dataset: {len(df):,} invocations")
print(f"Processed dataset: {len(df_features):,} invocations")
print(f"Features created: {df_features.shape[1]} columns")
print(f"Time series functions: {len(time_series_datasets)}")

print("\nKey Insights:")
print(f"Total functions analyzed: {df_features['func'].nunique():,}")
print(f"Average cold start rate: {df_features['is_potential_coldstart'].mean()*100:.1f}%")
print(f"Peak traffic hour: {df_features.groupby('hour').size().idxmax()}:00")
print(f"Most active day: {['Mon','Tue','Wed','Thu','Fri','Sat','Sun'][df_features.groupby('dayofweek').size().idxmax()]}")


In [ ]:
df_features.info()

Prophet Model Training and Testing

In [ ]:
df_243 = pd.read_csv('timeseries_func_243.csv')
print(df_243['y'].describe())
print(f"Zero traffic periods: {(df_243['y'] == 0).sum() / len(df_243) * 100:.1f}%")

In [ ]:
df_101 = pd.read_csv('timeseries_func_101.csv')
print(df_101['y'].describe())
print(f"Zero traffic periods: {(df_101['y'] == 0).sum() / len(df_101) * 100:.1f}%")

In [ ]:
FUNCTION_IDS = [
    'func_235', 'func_242', 'func_126', 'func_186',
     'func_190', 'func_110', 'func_99', 'func_191'
] #'func_243' and 'func_101' are excluded because of High zero-rate (80.9%) - reactive-only

TRAIN_RATIO = 0.8

In [ ]:
def train_prophet_model(df, func_id):

    # Fix data types
    df['ds'] = pd.to_datetime(df['ds'])
    df['y'] = pd.to_numeric(df['y'], errors='coerce')

    # Remove any NaN values
    df = df.dropna(subset=['ds', 'y'])

    # Train-test split
    split_idx = int(len(df) * TRAIN_RATIO)
    train = df.iloc[:split_idx][['ds', 'y']].copy()
    test = df.iloc[split_idx:][['ds', 'y']].copy()

    # Configure model
    model = Prophet(
        changepoint_prior_scale=0.05,
        seasonality_prior_scale=10.0,
        seasonality_mode='multiplicative',
        daily_seasonality=True,
        weekly_seasonality=True,
        yearly_seasonality=False,
        interval_width=0.95
    )

    # Train
    model.fit(train)

    # Predict
    forecast = model.predict(test[['ds']])

    # Calculate metrics
    y_true = test['y'].values
    y_pred = np.maximum(forecast['yhat'].values, 0)

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = np.mean(np.abs((y_true - y_pred) / (y_true + 1))) * 100

    # Detect anomalies
    forecast_test = forecast.set_index('ds')
    test_indexed = test.set_index('ds')
    anomalies = test_indexed[
        (test_indexed['y'] < forecast_test['yhat_lower']) |
        (test_indexed['y'] > forecast_test['yhat_upper'])
    ]

    return {
        'function': func_id,
        'train_size': len(train),
        'test_size': len(test),
        'mae': mae,
        'rmse': rmse,
        'mape': mape,
        'anomalies': len(anomalies),
        'anomaly_rate': len(anomalies) / len(test) * 100,
        'model': model,
        'forecast': forecast,
        'test': test
    }

print("Starting batch training for all functions...")

results = []

for func_id in tqdm(FUNCTION_IDS, desc="Training models"):
    try:
        # Load data with proper type handling
        df = pd.read_csv(f'timeseries_{func_id}.csv')

        # Convert data types explicitly
        df['ds'] = pd.to_datetime(df['ds'])
        df['y'] = pd.to_numeric(df['y'], errors='coerce')

        # Handle boolean columns if they exist
        for col in df.columns:
            if df[col].dtype == 'object':
                # Try to convert string booleans to numeric
                if df[col].str.lower().isin(['true', 'false']).any():
                    df[col] = df[col].str.lower().map({'true': 1, 'false': 0})

        # Drop any rows with NaN in critical columns
        df = df.dropna(subset=['ds', 'y'])

        if len(df) == 0:
            raise ValueError("No valid data after cleaning")

        # Train model
        result = train_prophet_model(df, func_id)
        results.append(result)

        print(f"\n {func_id}: MAE={result['mae']:.2f}, RMSE={result['rmse']:.2f}, MAPE={result['mape']:.1f}%")

    except FileNotFoundError:
        print(f"\n {func_id}: File not found - timeseries_{func_id}.csv")
    except ValueError as e:
        print(f"\n {func_id}: Data Issue - {str(e)}")
    except Exception as e:
        print(f"\n {func_id}: Error - {str(e)}")

print("\n")
print(f"Completed: {len(results)}/{len(FUNCTION_IDS)} functions")

# COMPARE RESULTS

# Create summary DataFrame
summary_df = pd.DataFrame([{
    'function': r['function'],
    'mae': r['mae'],
    'rmse': r['rmse'],
    'mape': r['mape'],
    'anomaly_rate': r['anomaly_rate']
} for r in results])

print("\n")
print("MODEL PERFORMANCE SUMMARY")
print(summary_df.to_string(index=False))
print("\nAverage metrics:")
print(f"  MAE:  {summary_df['mae'].mean():.2f}")
print(f"  RMSE: {summary_df['rmse'].mean():.2f}")
print(f"  MAPE: {summary_df['mape'].mean():.1f}%")

# Save summary
summary_df.to_csv('prophet_comparison_results.csv', index=False)

# VISUALIZE COMPARISON

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: MAE comparison
axes[0, 0].barh(summary_df['function'], summary_df['mae'], color='steelblue')
axes[0, 0].set_xlabel('Mean Absolute Error')
axes[0, 0].set_title('MAE by Function')
axes[0, 0].grid(axis='x', alpha=0.3)

# Plot 2: MAPE comparison
axes[0, 1].barh(summary_df['function'], summary_df['mape'], color='coral')
axes[0, 1].set_xlabel('Mean Absolute Percentage Error (%)')
axes[0, 1].set_title('MAPE by Function')
axes[0, 1].grid(axis='x', alpha=0.3)

# Plot 3: Anomaly rate
axes[1, 0].barh(summary_df['function'], summary_df['anomaly_rate'], color='indianred')
axes[1, 0].set_xlabel('Anomaly Rate (%)')
axes[1, 0].set_title('Anomaly Detection Rate')
axes[1, 0].grid(axis='x', alpha=0.3)

# Plot 4: MAE vs MAPE scatter
axes[1, 1].scatter(summary_df['mae'], summary_df['mape'], s=100, alpha=0.6, color='purple')
for idx, row in summary_df.iterrows():
    axes[1, 1].annotate(row['function'].split('_')[1],
                        (row['mae'], row['mape']),
                        fontsize=8, alpha=0.7)
axes[1, 1].set_xlabel('MAE')
axes[1, 1].set_ylabel('MAPE (%)')
axes[1, 1].set_title('MAE vs MAPE')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('prophet_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

# IDENTIFY BEST/WORST PERFORMERS

best_mae = summary_df.loc[summary_df['mae'].idxmin()]
worst_mae = summary_df.loc[summary_df['mae'].idxmax()]

print("\n")
print("BEST/WORST PERFORMERS")
print(f"Best (lowest MAE):  {best_mae['function']} - MAE={best_mae['mae']:.2f}")
print(f"Worst (highest MAE): {worst_mae['function']} - MAE={worst_mae['mae']:.2f}")



In [ ]:
# Map the exported function names back to original data
function_mapping = {
    'func_235': func_stats.iloc[0]['func'],  # Top function by invocation count
    'func_242': func_stats.iloc[1]['func'],
    'func_126': func_stats.iloc[2]['func'],
    'func_186': func_stats.iloc[4]['func'],
    'func_190': func_stats.iloc[6]['func'],
    'func_110': func_stats.iloc[7]['func'],
    'func_99': func_stats.iloc[8]['func'],
    'func_191': func_stats.iloc[9]['func']
}

# Create detailed profiles
detailed_profiles = []

for i in range(8):  # Excluding the 2 problematic ones
    func_name = f'func_{i+1}'
    original_func_id = func_stats.iloc[i]['func']
    app_id = func_stats.iloc[i]['app']

    # Get function data
    func_data = df_cp[(df_cp['app'] == app_id) & (df_cp['func'] == original_func_id)]

    if i < len(FUNCTION_IDS):
        func_name = FUNCTION_IDS[i] # Use the descriptive name
    else:
        func_name = f'func_{i+1}' # Fallback
    # ----------------------

    original_func_id = func_stats.iloc[i]['func']
    app_id = func_stats.iloc[i]['app']

    # Calculate detailed statistics
    profile = {
        'display_name': func_name,
        'original_id': original_func_id[:20] + '...',  # Truncated hash
        'app_id': app_id[:20] + '...',

        # Volume metrics
        'total_invocations': len(func_data),
        'invocations_per_hour': len(func_data) / func_stats.iloc[i]['activity_span_hours'],

        # Traffic density
        'zero_rate': (func_data['duration'] == 0).sum() / len(func_data) * 100,
        'non_zero_rate': 100 - ((func_data['duration'] == 0).sum() / len(func_data) * 100),

        # Performance characteristics
        'avg_duration_ms': func_data['duration'].mean() * 1000,
        'p95_duration_ms': func_data['duration'].quantile(0.95) * 1000,
        'duration_variability': func_data['duration'].std() / func_data['duration'].mean(),

        # Cold start analysis
        'cold_start_rate': func_stats.iloc[i]['cold_start_rate'] * 100,
        'total_cold_starts': func_stats.iloc[i]['cold_starts'],

        # Temporal patterns
        'activity_span_hours': func_stats.iloc[i]['activity_span_hours'],
        'peak_hour': func_data.groupby('hour').size().idxmax(),
        'peak_day': ['Mon','Tue','Wed','Thu','Fri','Sat','Sun'][
            func_data.groupby('dayofweek').size().idxmax()
        ],

        # Burstiness metrics
        'median_time_gap_sec': func_data['time_gap'].median(),
        'mean_time_gap_sec': func_data['time_gap'].mean(),
        'burst_invocations': ((func_data['time_gap'] <= 10) & func_data['time_gap'].notna()).sum(),
        'burst_rate': ((func_data['time_gap'] <= 10) & func_data['time_gap'].notna()).sum() / len(func_data) * 100,

        # Workload classification
        'workload_type': active_funcs[active_funcs['func'] == original_func_id]['workload_type'].values[0]
            if original_func_id in active_funcs['func'].values else 'Unknown'
    }

    detailed_profiles.append(profile)

# Convert to DataFrame for easy viewing
profiles_df = pd.DataFrame(detailed_profiles)

# Display
print("\n" + "="*100)
print("DETAILED FUNCTION PROFILES")
print("="*100)
print(profiles_df.to_string(index=False))

# Export
profiles_df.to_csv('function_detailed_profiles.csv', index=False)

Hybrid LSTM Model

In [ ]:
# LSTM Hyperparameters
LSTM_LOOKBACK = 30 # Use last 30 timesteps to predict next
LSTM_UNITS = 50 # memory cells in each LSTM layer
LSTM_LAYERS = 2 # Number of LSTM layers
DROPOUT_RATE = 0.2 # Dropout for regularization
EPOCHS = 100  #  Maximum number of times to train on the entire dataset
BATCH_SIZE = 32  #  Number of samples processed before updating model weights
VALIDATION_SPLIT = 0.2  #  20% of training data reserved for validation

In [ ]:
#Create sequences for LSTM training
def create_sequences(data, lookback):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i + lookback])
        y.append(data[i + lookback])
    return np.array(X), np.array(y)

In [ ]:
#Build LSTM model architecture
def build_lstm_model(lookback, units=50, layers=2, dropout=0.2):
    model = Sequential() #1-- create a model

    #2-- First LSTM layer
    model.add(LSTM(
        units=units,
        return_sequences=(layers > 1), #true
        input_shape=(lookback, 1)
    ))
    model.add(Dropout(dropout))

    # Additional LSTM layers
    for i in range(1, layers):
        model.add(LSTM(            #3-- second lstm layer
            units=units,
            return_sequences=(i < layers - 1) #false
        ))
        model.add(Dropout(dropout))   #4-- dropping some units to avoid mistakes

    # Output layer
    model.add(Dense(1))     #5-- dense layer-for decision making, what to do with the processing, 1-because only 1 output number to predict

    model.compile(optimizer='adam', loss='mse', metrics=['mae']) #compile to tell the model how to improve

    return model

In [ ]:
#Calculate MAPE, handling zero values
def calculate_mape(y_true, y_pred):
    mask = y_true != 0
    if mask.sum() == 0:
        return np.nan
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100


Train LSTM on Prophet residuals

In [ ]:
def train_lstm_on_residuals(df, func_id, prophet_result):  #returns dict with LSTM model and metrics
    print(f"\n")
    print(f"Training LSTM for {func_id}")
    print(f"\n")

    # Getting Prophet's test predictions
    test_data = prophet_result['test']
    forecast = prophet_result['forecast']

    # Aligning forecast with test data
    forecast_aligned = forecast.set_index('ds').loc[test_data['ds']]

    # Calculating residuals (Prophet's errors)
    residuals = test_data['y'].values - forecast_aligned['yhat'].values

    print(f"Residual statistics:")
    print(f"  Mean: {residuals.mean():.2f}")
    print(f"  Std:  {residuals.std():.2f}")
    print(f"  Min:  {residuals.min():.2f}")
    print(f"  Max:  {residuals.max():.2f}")

    # Checking if we have enough data
    if len(residuals) < LSTM_LOOKBACK + 50:  # Need minimum data for LSTM
        print(f"  Insufficient data for LSTM (need >{LSTM_LOOKBACK + 50}, have {len(residuals)})")
        return None

    # Create sequences for LSTM
    X, y = create_sequences(residuals, LSTM_LOOKBACK)

    print(f"LSTM sequences created: {len(X)} samples")

    # Split into train and validation
    split_idx = int(len(X) * (1 - VALIDATION_SPLIT))
    X_train, X_val = X[:split_idx], X[split_idx:]
    y_train, y_val = y[:split_idx], y[split_idx:]

    # Scale the data
    X_scaler = MinMaxScaler()
    y_scaler = MinMaxScaler()

    X_train_scaled = X_scaler.fit_transform(X_train.reshape(-1, 1)).reshape(X_train.shape[0], LSTM_LOOKBACK, 1)
    X_val_scaled = X_scaler.transform(X_val.reshape(-1, 1)).reshape(X_val.shape[0], LSTM_LOOKBACK, 1)

    y_train_scaled = y_scaler.fit_transform(y_train.reshape(-1, 1))
    y_val_scaled = y_scaler.transform(y_val.reshape(-1, 1))

    # Build and train LSTM
    lstm_model = build_lstm_model(
        lookback=LSTM_LOOKBACK,
        units=LSTM_UNITS,
        layers=LSTM_LAYERS,
        dropout=DROPOUT_RATE
    )

    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=0
    )

    print("Training LSTM...")
    history = lstm_model.fit(
        X_train_scaled, y_train_scaled,
        validation_data=(X_val_scaled, y_val_scaled),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[early_stopping],
        verbose=0
    )

    print(f" Training completed:")
    print(f"  Epochs: {len(history.history['loss'])}")
    print(f"  Final train loss: {history.history['loss'][-1]:.4f}")
    print(f"  Final val loss: {history.history['val_loss'][-1]:.4f}")

    # Predicting residuals on validation set
    y_pred_scaled = lstm_model.predict(X_val_scaled, verbose=0)
    y_pred = y_scaler.inverse_transform(y_pred_scaled).flatten()

    # Calculating LSTM residual prediction accuracy
    lstm_mae = mean_absolute_error(y_val, y_pred)
    lstm_rmse = np.sqrt(mean_squared_error(y_val, y_pred))

    print(f"LSTM residual prediction:")
    print(f"  MAE:  {lstm_mae:.2f}")
    print(f"  RMSE: {lstm_rmse:.2f}")

    return {
        'lstm_model': lstm_model,
        'X_scaler': X_scaler,
        'y_scaler': y_scaler,
        'history': history,
        'lstm_mae': lstm_mae,
        'lstm_rmse': lstm_rmse,
        'residuals': residuals,
        'X_val': X_val,
        'y_val': y_val,
        'y_pred': y_pred
    }


Evaluating Hybrid results

In [ ]:
def evaluate_hybrid_model(prophet_result, lstm_result, func_id):
    if lstm_result is None:
        return None

    print(f"\n")
    print(f"Evaluating Hybrid Model for {func_id}")
    print(f"\n")

    # Get test data and Prophet predictions
    test_data = prophet_result['test']
    forecast = prophet_result['forecast']
    forecast_aligned = forecast.set_index('ds').loc[test_data['ds']]

    prophet_pred = forecast_aligned['yhat'].values
    actual = test_data['y'].values

    # Get LSTM residual predictions
    residuals = lstm_result['residuals']
    X_val = lstm_result['X_val']

    # Predict residuals with LSTM for the entire test set
    X_full, _ = create_sequences(residuals, LSTM_LOOKBACK)
    X_full_scaled = lstm_result['X_scaler'].transform(X_full.reshape(-1, 1)).reshape(X_full.shape[0], LSTM_LOOKBACK, 1)

    lstm_residual_pred_scaled = lstm_result['lstm_model'].predict(X_full_scaled, verbose=0)
    lstm_residual_pred = lstm_result['y_scaler'].inverse_transform(lstm_residual_pred_scaled).flatten()

    # Align predictions (LSTM starts after lookback period)
    prophet_pred_aligned = prophet_pred[LSTM_LOOKBACK:]
    actual_aligned = actual[LSTM_LOOKBACK:]

    # Create hybrid predictions
    hybrid_pred = prophet_pred_aligned + lstm_residual_pred
    hybrid_pred = np.maximum(hybrid_pred, 0)  # No negative predictions

    # Calculate metrics for Prophet-only
    prophet_mae = mean_absolute_error(actual_aligned, prophet_pred_aligned)
    prophet_rmse = np.sqrt(mean_squared_error(actual_aligned, prophet_pred_aligned))
    prophet_mape = calculate_mape(actual_aligned, prophet_pred_aligned)

    # Calculate metrics for Hybrid
    hybrid_mae = mean_absolute_error(actual_aligned, hybrid_pred)
    hybrid_rmse = np.sqrt(mean_squared_error(actual_aligned, hybrid_pred))
    hybrid_mape = calculate_mape(actual_aligned, hybrid_pred)

    # Calculate improvement
    mae_improvement = ((prophet_mae - hybrid_mae) / prophet_mae) * 100
    rmse_improvement = ((prophet_rmse - hybrid_rmse) / prophet_rmse) * 100

    print(f"\nProphet-Only Performance:")
    print(f"  MAE:  {prophet_mae:.2f}")
    print(f"  RMSE: {prophet_rmse:.2f}")
    print(f"  MAPE: {prophet_mape:.2f}%")

    print(f"\nHybrid Model Performance:")
    print(f"  MAE:  {hybrid_mae:.2f}")
    print(f"  RMSE: {hybrid_rmse:.2f}")
    print(f"  MAPE: {hybrid_mape:.2f}%")

    print(f"\nImprovement:")
    print(f"  MAE:  {mae_improvement:+.2f}%")
    print(f"  RMSE: {rmse_improvement:+.2f}%")

    if mae_improvement > 0:
        print(f"\n Hybrid model improved by {mae_improvement:.1f}%!")
    else:
        print(f"\n  Prophet alone performs better (degradation: {mae_improvement:.1f}%)")

    return {
        'function': func_id,
        'prophet_mae': prophet_mae,
        'prophet_rmse': prophet_rmse,
        'prophet_mape': prophet_mape,
        'hybrid_mae': hybrid_mae,
        'hybrid_rmse': hybrid_rmse,
        'hybrid_mape': hybrid_mape,
        'mae_improvement': mae_improvement,
        'rmse_improvement': rmse_improvement,
        'prophet_pred': prophet_pred_aligned,
        'hybrid_pred': hybrid_pred,
        'actual': actual_aligned,
        'test_dates': test_data['ds'].values[LSTM_LOOKBACK:]
    }


Visualization

In [ ]:
#Plot comparison of Prophet vs Hybrid predictions
def plot_hybrid_comparison(eval_result, func_id, save_path=None):
    if eval_result is None:
        return

    fig, axes = plt.subplots(2, 1, figsize=(15, 10))

    dates = pd.to_datetime(eval_result['test_dates'])
    actual = eval_result['actual']
    prophet_pred = eval_result['prophet_pred']
    hybrid_pred = eval_result['hybrid_pred']

    # Plot 1: Predictions comparison
    ax1 = axes[0]
    ax1.plot(dates, actual, label='Actual', color='black', linewidth=2, alpha=0.7)
    ax1.plot(dates, prophet_pred, label='Prophet Only', color='blue', linewidth=1.5, alpha=0.7)
    ax1.plot(dates, hybrid_pred, label='Hybrid (Prophet+LSTM)', color='red', linewidth=1.5, alpha=0.7)

    ax1.set_xlabel('Time', fontsize=12)
    ax1.set_ylabel('Requests per Minute', fontsize=12)
    ax1.set_title(f'{func_id}: Hybrid Model Predictions', fontsize=14, fontweight='bold')
    ax1.legend(loc='upper left', fontsize=10)
    ax1.grid(True, alpha=0.3)

    # Plot 2: Residuals comparison
    ax2 = axes[1]
    prophet_residuals = actual - prophet_pred
    hybrid_residuals = actual - hybrid_pred

    ax2.plot(dates, prophet_residuals, label='Prophet Residuals', color='blue', alpha=0.6)
    ax2.plot(dates, hybrid_residuals, label='Hybrid Residuals', color='red', alpha=0.6)
    ax2.axhline(y=0, color='black', linestyle='--', alpha=0.5)

    ax2.set_xlabel('Time', fontsize=12)
    ax2.set_ylabel('Residual (Actual - Predicted)', fontsize=12)
    ax2.set_title('Prediction Residuals', fontsize=14, fontweight='bold')
    ax2.legend(loc='upper left', fontsize=10)
    ax2.grid(True, alpha=0.3)

    # Add improvement text
    improvement_text = f"MAE Improvement: {eval_result['mae_improvement']:+.1f}%"
    ax1.text(0.02, 0.98, improvement_text,
             transform=ax1.transAxes,
             fontsize=11,
             verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f" Plot saved: {save_path}")

    plt.show()

Batch processing by hybrid model

In [ ]:
def process_all_functions(results): #existing prophet results
    print("\n")
    print(" HYBRID PROPHET-LSTM BATCH TRAINING")
    print("\n")

    hybrid_results = []
    lstm_models = {}

    for prophet_result in tqdm(results, desc="Training LSTM models"):
        func_id = prophet_result['function']

        try:
            # Load original time series data
            df = pd.read_csv(f'timeseries_{func_id}.csv')
            df['ds'] = pd.to_datetime(df['ds'])
            df['y'] = pd.to_numeric(df['y'], errors='coerce')
            df = df.dropna(subset=['ds', 'y'])

            # Train LSTM on residuals
            lstm_result = train_lstm_on_residuals(df, func_id, prophet_result)

            if lstm_result is None:
                print(f"  Skipping {func_id} - insufficient data")
                continue

            # Evaluate hybrid model
            eval_result = evaluate_hybrid_model(prophet_result, lstm_result, func_id)

            if eval_result is not None:
                hybrid_results.append(eval_result)
                lstm_models[func_id] = lstm_result

                # Plot comparison
                plot_hybrid_comparison(eval_result, func_id,
                                     save_path=f'hybrid_{func_id}_prediction.png')

        except Exception as e:
            print(f" Error processing {func_id}: {str(e)}")
            continue

    return hybrid_results, lstm_models

In [ ]:
#Summary report
def generate_summary_report(hybrid_results):
    if len(hybrid_results) == 0:
        print("No results to summarize.")
        return

    # Create DataFrame
    summary_df = pd.DataFrame(hybrid_results)

    print("\n")
    print(" HYBRID MODEL SUMMARY REPORT")
    print("\n")

    # Display results table
    print("\nDetailed Results:")
    print(summary_df[['function', 'prophet_mae', 'hybrid_mae', 'mae_improvement',
                      'prophet_rmse', 'hybrid_rmse', 'rmse_improvement']].to_string(index=False))

    # Overall statistics
    print("\n")
    print("Overall Performance:")
    print("\n")
    print(f"\nProphet Average:")
    print(f"  MAE:  {summary_df['prophet_mae'].mean():.2f} (±{summary_df['prophet_mae'].std():.2f})")
    print(f"  RMSE: {summary_df['prophet_rmse'].mean():.2f} (±{summary_df['prophet_rmse'].std():.2f})")
    print(f"  MAPE: {summary_df['prophet_mape'].mean():.2f}%")

    print(f"\nHybrid Average:")
    print(f"  MAE:  {summary_df['hybrid_mae'].mean():.2f} (±{summary_df['hybrid_mae'].std():.2f})")
    print(f"  RMSE: {summary_df['hybrid_rmse'].mean():.2f} (±{summary_df['hybrid_rmse'].std():.2f})")
    print(f"  MAPE: {summary_df['hybrid_mape'].mean():.2f}%")

    print(f"\nAverage Improvement:")
    print(f"  MAE:  {summary_df['mae_improvement'].mean():+.2f}%")
    print(f"  RMSE: {summary_df['rmse_improvement'].mean():+.2f}%")

    # Count improvements
    improved = (summary_df['mae_improvement'] > 0).sum()
    total = len(summary_df)

    print(f"\nFunctions Improved: {improved}/{total} ({improved/total*100:.1f}%)")

    # Save results
    summary_df.to_csv('hybrid_model_results.csv', index=False)
    print(f"\n Results saved to: hybrid_model_results.csv")

    # Create comparison visualization
    create_summary_plots(summary_df)

    return summary_df

In [ ]:
#Create summary comparison plots
def create_summary_plots(summary_df):
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))

    x = np.arange(len(summary_df))
    width = 0.35

    # Plot 1: MAE Comparison
    ax1 = axes[0, 0]
    ax1.bar(x - width/2, summary_df['prophet_mae'], width,
            label='Prophet', color='blue', alpha=0.7)
    ax1.bar(x + width/2, summary_df['hybrid_mae'], width,
            label='Hybrid', color='red', alpha=0.7)
    ax1.set_xlabel('Function')
    ax1.set_ylabel('MAE')
    ax1.set_title('MAE: Prophet vs Hybrid', fontweight='bold')
    ax1.set_xticks(x)
    ax1.set_xticklabels([f.replace('func_', 'F') for f in summary_df['function']], rotation=45)
    ax1.legend()
    ax1.grid(True, alpha=0.3, axis='y')

    # Plot 2: RMSE Comparison
    ax2 = axes[0, 1]
    ax2.bar(x - width/2, summary_df['prophet_rmse'], width,
            label='Prophet', color='blue', alpha=0.7)
    ax2.bar(x + width/2, summary_df['hybrid_rmse'], width,
            label='Hybrid', color='red', alpha=0.7)
    ax2.set_xlabel('Function')
    ax2.set_ylabel('RMSE')
    ax2.set_title('RMSE: Prophet vs Hybrid', fontweight='bold')
    ax2.set_xticks(x)
    ax2.set_xticklabels([f.replace('func_', 'F') for f in summary_df['function']], rotation=45)
    ax2.legend()
    ax2.grid(True, alpha=0.3, axis='y')

    # Plot 3: Improvement Percentage
    ax3 = axes[1, 0]
    colors = ['green' if val > 0 else 'red' for val in summary_df['mae_improvement']]
    ax3.bar(x, summary_df['mae_improvement'], color=colors, alpha=0.7)
    ax3.axhline(y=0, color='black', linestyle='--', linewidth=1)
    ax3.set_xlabel('Function')
    ax3.set_ylabel('MAE Improvement (%)')
    ax3.set_title('Hybrid Improvement over Prophet', fontweight='bold')
    ax3.set_xticks(x)
    ax3.set_xticklabels([f.replace('func_', 'F') for f in summary_df['function']], rotation=45)
    ax3.grid(True, alpha=0.3, axis='y')

    # Plot 4: Scatter
    ax4 = axes[1, 1]
    ax4.scatter(summary_df['prophet_mae'], summary_df['hybrid_mae'],
                s=100, alpha=0.6, c='purple')
    max_val = max(summary_df['prophet_mae'].max(), summary_df['hybrid_mae'].max())
    ax4.plot([0, max_val], [0, max_val], 'k--', linewidth=1, alpha=0.5)
    ax4.set_xlabel('Prophet MAE')
    ax4.set_ylabel('Hybrid MAE')
    ax4.set_title('Prophet vs Hybrid MAE\n(Below diagonal = Hybrid better)', fontweight='bold')
    ax4.grid(True, alpha=0.3)

    improved = (summary_df['hybrid_mae'] < summary_df['prophet_mae']).sum()
    ax4.text(0.05, 0.95, f'{improved}/{len(summary_df)} improved',
             transform=ax4.transAxes,
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

    plt.tight_layout()
    plt.savefig('hybrid_summary_comparison.png', dpi=300, bbox_inches='tight')
    print(" Summary plots saved: hybrid_summary_comparison.png")
    plt.show()


In [ ]:
print(" Starting Hybrid LSTM Training...")
if 'results' not in locals():
    print(" ERROR: 'results' variable not found")
    print("Run Prophet training code first")
else:
    print(f" Found {len(results)} Prophet results")
    print("\nProcessing functions with LSTM...\n")

    # Train LSTM on all functions
    hybrid_results, lstm_models = process_all_functions(results)

    # Generate summary report
    if len(hybrid_results) > 0:
        print("\n" )
        print("Generating Summary Report...")
        summary_df = generate_summary_report(hybrid_results)

        print("\n DONE! ")
    else:
        print("\n No hybrid results generated. Check errors.")

In [ ]:


'''# Testing lookback values
lookback_values = [15, 30, 45, 60]

for LSTM_LOOKBACK in lookback_values:
    print(f"\nTesting LSTM_LOOKBACK = {LSTM_LOOKBACK}")

    func_id = 'func_110'
    df = pd.read_csv(f'timeseries_{func_id}.csv')
    df['ds'] = pd.to_datetime(df['ds'])
    df['y'] = pd.to_numeric(df['y'])
    df = df.dropna()

    prophet_result = [r for r in results if r['function'] == func_id][0]

    # This uses your existing function with the updated LSTM_LOOKBACK
    lstm_result = train_lstm_on_residuals(df, func_id, prophet_result)

    if lstm_result:
        eval_result = evaluate_hybrid_model(prophet_result, lstm_result, func_id)
        if eval_result:
            print(f"\n Lookback {LSTM_LOOKBACK}: MAE = {eval_result['hybrid_mae']:.2f}, "
                  f"Improvement = {eval_result['mae_improvement']:+.1f}%")'''

In [ ]:
# Test lookback values
lookback_values = [15, 30, 45, 60]

for LSTM_LOOKBACK in lookback_values:
    print(f"\nTesting LSTM_LOOKBACK = {LSTM_LOOKBACK}")

    func_id = 'func_235'
    df = pd.read_csv(f'timeseries_{func_id}.csv')
    df['ds'] = pd.to_datetime(df['ds'])
    df['y'] = pd.to_numeric(df['y'])
    df = df.dropna()

    prophet_result = [r for r in results if r['function'] == func_id][0]

    # This uses your existing function with the updated LSTM_LOOKBACK
    lstm_result = train_lstm_on_residuals(df, func_id, prophet_result)

    if lstm_result:
        eval_result = evaluate_hybrid_model(prophet_result, lstm_result, func_id)
        if eval_result:
            print(f"\n Lookback {LSTM_LOOKBACK}: MAE = {eval_result['hybrid_mae']:.2f}, "
                  f"Improvement = {eval_result['mae_improvement']:+.1f}%")

LSTM only model training

In [ ]:
# Use same configuration as hybrid
LSTM_LOOKBACK = 30
LSTM_UNITS = 50
LSTM_LAYERS = 2
DROPOUT_RATE = 0.2
EPOCHS = 100
BATCH_SIZE = 32
VALIDATION_SPLIT = 0.2

In [ ]:
def train_lstm_only(df, func_id):
  print(f"Training LSTM-only for {func_id}")
  #prepare data
  df = df.copy()
  df['ds'] = pd.to_datetime(df['ds'])
  df['y'] = pd.to_numeric(df['y'], errors = 'coerce')
  df = df.dropna(subset = ['ds','y'])

  print(f"Total data points = {len(df):,}")
  train_size = int(len(df)*0.8)
  train_df = df[:train_size]
  test_df = df[train_size:]

  print(f"Train size: {len(train_df):,}")
  print(f"Test size: {len(test_df):,}")

  #Extract values
  train_data = train_df['y']
  test_data = test_df['y']

  #create sequences
  print(f"\nCreating sequence with lookback value: {LSTM_LOOKBACK}...")

  X_train, y_train = create_sequences(train_data, LSTM_LOOKBACK)

  if len(X_train) < 50:
    print(f"Insufficient training data only {len(X_train)} samples")
    return None

  print(f"Training sequences {len(X_train):,}")

  #Split train into train and validation

  split_idx = int(len(X_train)* (1-VALIDATION_SPLIT))
  X_train_final = X_train[:split_idx]
  x_val = X_train[split_idx:]
  y_train_final = y_train[:split_idx]
  y_val = y_train[split_idx:]

  #Scale the data
  X_scaler = MinMaxScaler()
  y_scaler = MinMaxScaler()

  X_train_scaled = X_scaler.fit_transform(X_train_final.reshape(-1,1)).reshape(X_train_final.shape[0], LSTM_LOOKBACK, 1)
  x_val_scaled = X_scaler.transform(x_val.reshape(-1,1)).reshape(x_val.shape[0], LSTM_LOOKBACK, 1)

  y_train_scaled = y_scaler.fit_transform(y_train_final.reshape(-1,1))
  y_val_scaled = y_scaler.transform(y_val.reshape(-1,1))

  #Build LSTM model
  print("\nBuilding LSTM model")
  print(f"LSTM Architecture: {LSTM_LAYERS} layers, {LSTM_UNITS} units, {DROPOUT_RATE} dropout")

  model = Sequential()

  #Adding first layer
  model.add(LSTM(
      units = LSTM_UNITS,
      return_sequences = (LSTM_LAYERS>1),
      input_shape = (LSTM_LOOKBACK,1)
  ))

  model.add(Dropout(DROPOUT_RATE))

  #Additional LSTM layers

  for i in range(1, LSTM_LAYERS):
    model.add(LSTM(
        units = LSTM_UNITS,
        return_sequences = (i < LSTM_LAYERS - 1)
    ))

  model.add(Dropout(DROPOUT_RATE))

  #output layer
  model.add(Dense(1))

  model.compile(optimizer='adam', loss = 'mse', metrics = ['mae'])

  #Train
  print("Training LSTM...")

  early_stopping = EarlyStopping(
      monitor = 'val_loss',
      patience = 10,
      restore_best_weights = True,
      verbose = 0
  )

  history = model.fit(
    X_train_scaled, y_train_scaled,
    validation_data = (x_val_scaled, y_val_scaled),
    epochs = EPOCHS,
    batch_size = BATCH_SIZE,
    callbacks = [early_stopping],
    verbose = 0
  )

  print(f" Training completed:")
  print(f"  Epochs: {len(history.history['loss'])}")
  print(f"  Final train loss: {history.history['loss'][-1]:.4f}")
  print(f"  Final val loss: {history.history['val_loss'][-1]:.4f}")

  # Prepare test sequences
  print(f"\nPreparing test predictions...")

  # Use last LSTM_LOOKBACK points from training as initial context
  test_data_with_context = np.concatenate([train_data[-LSTM_LOOKBACK:], test_data])

  # Create test sequences
  X_test, y_test = create_sequences(test_data_with_context, LSTM_LOOKBACK)

  # The first len(test_data) predictions correspond to our test set
  X_test = X_test[:len(test_data)]
  y_test = y_test[:len(test_data)]

  # Scale test data
  X_test_scaled = X_scaler.transform(X_test.reshape(-1, 1)).reshape(
    X_test.shape[0], LSTM_LOOKBACK, 1)

  # Predict
  y_pred_scaled = model.predict(X_test_scaled, verbose=0)
  y_pred = y_scaler.inverse_transform(y_pred_scaled).flatten()

  # Ensure non-negative predictions
  y_pred = np.maximum(y_pred, 0)

  # Calculate metrics
  mae = mean_absolute_error(y_test, y_pred)
  rmse = np.sqrt(mean_squared_error(y_test, y_pred))

  # Calculate MAPE
  mask = y_test != 0
  if mask.sum() > 0:
    mape = np.mean(np.abs((y_test[mask] - y_pred[mask]) / y_test[mask])) * 100
  else:
     mape = np.nan

  print(f"\nLSTM-Only Test Performance:")
  print(f"  MAE:  {mae:.2f}")
  print(f"  RMSE: {rmse:.2f}")
  print(f"  MAPE: {mape:.2f}%")

  return {
        'model': model,
        'X_scaler': X_scaler,
        'y_scaler': y_scaler,
        'history': history,
        'mae': mae,
        'rmse': rmse,
        'mape': mape,
        'y_test': y_test,
        'y_pred': y_pred,
        'test_dates': test_df['ds'].values[:len(y_pred)]
    }


In [ ]:
def compare_all_models(func_id, prophet_result, lstm_only_result, hybrid_result):
  print(f"THREE-WAY MODEL COMPARISON: {func_id}")

  # Get Prophet metrics
  test_data = prophet_result['test']
  forecast = prophet_result['forecast']
  prophet_pred = forecast['yhat'].values
  actual = test_data['y'].values

  prophet_mae = mean_absolute_error(actual, prophet_pred)
  prophet_rmse = np.sqrt(mean_squared_error(actual, prophet_pred))

  # LSTM-only metrics (already calculated)
  lstm_mae = lstm_only_result['mae']
  lstm_rmse = lstm_only_result['rmse']

  # Hybrid metrics (already calculated)
  hybrid_mae = hybrid_result['hybrid_mae']
  hybrid_rmse = hybrid_result['hybrid_rmse']

  print("\n PERFORMANCE COMPARISON")

  print(f"{'Model':<20} {'MAE':<12} {'RMSE':<12} {'vs Prophet':<15}")

  print(f"{'Prophet-only':<20} {prophet_mae:<12.2f} {prophet_rmse:<12.2f} {'(baseline)':<15}")

  lstm_vs_prophet = ((prophet_mae - lstm_mae) / prophet_mae) * 100
  print(f"{'LSTM-only':<20} {lstm_mae:<12.2f} {lstm_rmse:<12.2f} {lstm_vs_prophet:+.1f}%")

  hybrid_vs_prophet = ((prophet_mae - hybrid_mae) / prophet_mae) * 100
  print(f"{'Hybrid (P+L)':<20} {hybrid_mae:<12.2f} {hybrid_rmse:<12.2f} {hybrid_vs_prophet:+.1f}%")


  # Determine best model
  best_mae = min(prophet_mae, lstm_mae, hybrid_mae)

  if best_mae == prophet_mae:
      best_model = "Prophet-only"
  elif best_mae == lstm_mae:
      best_model = "LSTM-only"
  else:
      best_model = "Hybrid"

  print(f"\ Best Model: {best_model} (MAE: {best_mae:.2f})")

  # Additional comparisons
  print(f"\n RELATIVE PERFORMANCE")

  if lstm_mae < prophet_mae:
      print(f" LSTM-only beats Prophet by {lstm_vs_prophet:.1f}%")
  else:
      print(f" LSTM-only worse than Prophet by {-lstm_vs_prophet:.1f}%")

  if hybrid_mae < prophet_mae:
      print(f" Hybrid beats Prophet by {hybrid_vs_prophet:.1f}%")
  else:
      print(f" Hybrid worse than Prophet by {-hybrid_vs_prophet:.1f}%")

  if hybrid_mae < lstm_mae:
      lstm_vs_hybrid = ((lstm_mae - hybrid_mae) / lstm_mae) * 100
      print(f" Hybrid beats LSTM-only by {lstm_vs_hybrid:.1f}%")
  else:
      lstm_vs_hybrid = ((lstm_mae - hybrid_mae) / lstm_mae) * 100
      print(f" Hybrid worse than LSTM-only by {-lstm_vs_hybrid:.1f}%")

  return {
        'function': func_id,
        'prophet_mae': prophet_mae,
        'prophet_rmse': prophet_rmse,
        'lstm_mae': lstm_mae,
        'lstm_rmse': lstm_rmse,
        'hybrid_mae': hybrid_mae,
        'hybrid_rmse': hybrid_rmse,
        'best_model': best_model,
        'prophet_vs_lstm': lstm_vs_prophet,
        'prophet_vs_hybrid': hybrid_vs_prophet,
        'lstm_vs_hybrid': lstm_vs_hybrid
  }

In [ ]:
def plot_three_way_comparison(func_id, prophet_result, lstm_only_result, hybrid_result):
    """Create visualization comparing all three models."""

    fig, axes = plt.subplots(2, 1, figsize=(15, 10))

    # Get data
    test_data = prophet_result['test']
    actual = test_data['y'].values
    dates = test_data['ds'].values

    prophet_pred = prophet_result['forecast']['yhat'].values

    # LSTM predictions (need to align)
    lstm_pred = lstm_only_result['y_pred']
    lstm_dates = lstm_only_result['test_dates']

    # Hybrid predictions (already aligned)
    hybrid_pred = hybrid_result['hybrid_pred']
    hybrid_dates = hybrid_result['test_dates']

    # Plot 1: All predictions
    ax1 = axes[0]
    ax1.plot(dates, actual, label='Actual', color='black', linewidth=2, alpha=0.8)
    ax1.plot(dates, prophet_pred, label='Prophet', color='blue', linewidth=1.5, alpha=0.7)
    ax1.plot(lstm_dates, lstm_pred, label='LSTM-only', color='green', linewidth=1.5, alpha=0.7)
    ax1.plot(hybrid_dates, hybrid_pred, label='Hybrid', color='red', linewidth=1.5, alpha=0.7)

    ax1.set_xlabel('Time', fontsize=12)
    ax1.set_ylabel('Requests per Minute', fontsize=12)
    ax1.set_title(f'{func_id}: Three-Way Model Comparison', fontsize=14, fontweight='bold')
    ax1.legend(loc='upper left', fontsize=10)
    ax1.grid(True, alpha=0.3)

    # Plot 2: Residuals
    ax2 = axes[1]
    prophet_residuals = actual - prophet_pred
    lstm_residuals = lstm_only_result['y_test'] - lstm_pred
    hybrid_residuals = hybrid_result['actual'] - hybrid_pred

    ax2.plot(dates, prophet_residuals, label='Prophet Residuals', color='blue', alpha=0.6)
    ax2.plot(lstm_dates, lstm_residuals, label='LSTM Residuals', color='green', alpha=0.6)
    ax2.plot(hybrid_dates, hybrid_residuals, label='Hybrid Residuals', color='red', alpha=0.6)
    ax2.axhline(y=0, color='black', linestyle='--', alpha=0.5)

    ax2.set_xlabel('Time', fontsize=12)
    ax2.set_ylabel('Residual (Actual - Predicted)', fontsize=12)
    ax2.set_title('Prediction Residuals', fontsize=14, fontweight='bold')
    ax2.legend(loc='upper left', fontsize=10)
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f'three_way_comparison_{func_id}.png', dpi=300, bbox_inches='tight')
    print(f"\n Plot saved: three_way_comparison_{func_id}.png")
    plt.show()

In [ ]:
def process_lstm_only_all_functions(results):
    """Train LSTM-only model on all functions."""

    print("\n")
    print("BATCH LSTM-ONLY TRAINING")

    lstm_only_results = []

    for prophet_result in results:
        func_id = prophet_result['function']

        print(f"\nProcessing {func_id}...")

        try:
            # Load data
            df = pd.read_csv(f'timeseries_{func_id}.csv')

            # Train LSTM-only
            lstm_result = train_lstm_only(df, func_id)

            if lstm_result:
                lstm_only_results.append({
                    'function': func_id,
                    'lstm_mae': lstm_result['mae'],
                    'lstm_rmse': lstm_result['rmse'],
                    'lstm_mape': lstm_result['mape'],
                    'lstm_result': lstm_result
                })

        except Exception as e:
            print(f" Error processing {func_id}: {str(e)}")
            continue

    return lstm_only_results

In [ ]:
def full_three_way_comparison(prophet_results, hybrid_results, lstm_only_results):
    """Create comprehensive comparison of all three approaches."""

    print("\n" )
    print("COMPLETE THREE-WAY MODEL COMPARISON")

    comparison_results = []

    for func_id in [r['function'] for r in prophet_results]:
        print(f"\n{'='*80}")
        print(f"Function: {func_id}")
        print(f"{'='*80}")

        # Get results for this function
        prophet_res = [r for r in prophet_results if r['function'] == func_id][0]
        hybrid_res = [r for r in hybrid_results if r['function'] == func_id]
        lstm_res = [r for r in lstm_only_results if r['function'] == func_id]

        if not hybrid_res or not lstm_res:
            print(f"  Missing results for {func_id}, skipping...")
            continue

        hybrid_res = hybrid_res[0]
        lstm_res = lstm_res[0]

        # Compare
        comparison = compare_all_models(
            func_id,
            prophet_res,
            lstm_res['lstm_result'],
            hybrid_res
        )

        comparison_results.append(comparison)

        # Plot
        try:
            plot_three_way_comparison(
                func_id,
                prophet_res,
                lstm_res['lstm_result'],
                hybrid_res
            )
        except Exception as e:
            print(f"  Could not create plot: {str(e)}")

    # Create summary
    comparison_df = pd.DataFrame(comparison_results)

    print("\n" )
    print("OVERALL SUMMARY")


    print("\n Average Performance:")
    print(f"  Prophet MAE:  {comparison_df['prophet_mae'].mean():.2f}")
    print(f"  LSTM MAE:     {comparison_df['lstm_mae'].mean():.2f}")
    print(f"  Hybrid MAE:   {comparison_df['hybrid_mae'].mean():.2f}")

    print("\n Best Model by Function:")
    best_model_counts = comparison_df['best_model'].value_counts()
    for model, count in best_model_counts.items():
        percentage = (count / len(comparison_df)) * 100
        print(f"  {model}: {count}/{len(comparison_df)} ({percentage:.1f}%)")

    # Save results
    comparison_df.to_csv('three_way_model_comparison.csv', index=False)
    print(f"\n Results saved to: three_way_model_comparison.csv")

    return comparison_df


In [ ]:
lstm_only_results = process_lstm_only_all_functions(results)

In [ ]:
comparison_df = full_three_way_comparison(
    prophet_results=results,          # Prophet results
    hybrid_results=hybrid_results,    # hybrid results
    lstm_only_results=lstm_only_results
)

In [ ]:
print(comparison_df[['function', 'prophet_mae', 'lstm_mae', 'hybrid_mae', 'best_model']])